## Imports

In [ ]:
import sys
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.utils.lensing import LensModel
from gleam.reconsrc import ReconSrc, run_model
from gleam.utils.plotting import kappa_map_plot, kappa_profiles_plot
from gleam.utils.plotting import arrival_time_surface_plot
from gleam.utils.plotting import plot_scalebar, plot_labelbox
from gleam.utils.rgb_map import radial_mask
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()


## Load data

In [ ]:
#        ACS          ACS(WFC3)        ACS(WFC3)      WFC3(UNK)
#        WFC3         SBC              WFC3           ACS(GAIA)
objs = ['B1608+656', 'DESJ0408-5354', 'HE0435-1223', 'PG1115+080',
        'RXJ0911+0551', 'RXJ1131-1231', 'SDSSJ1004+4112', 'WFIJ2033-4723']
objidx = 0
sigf = 20     #   80(20),  100(25),   60(12),  600(60),  140(35), 4000 (50),   80,   80
lens = objs[objidx]
print(lens)


In [ ]:
fitsdir = 'data/delay_qsos/'
fitsfile = fitsdir + '{}.fits'.format(lens)
print(fitsfile)

jsondir = 'jsons/'
jsonfile = jsondir + '{}.json'.format(lens)
print(jsonfile)


In [ ]:
with open(jsonfile) as f:
    lo = LensObject.from_json(f)
lo.squarify(0)   # 0, 0, 0, 0, 0
print(lo.__v__)


In [ ]:
statedir = 'states/'
statefiles = ['11doubles_dg45.state',
              '11doubles_dg60.state', '11doubles_CMB_dg60.state', '11doubles_SNeIa_dg60.state',
              '7quads_dg45.state', '7quads_CMB_dg45.state', '7quads_SNeIa_dg45.state',
              '7quads_dg60.state', '7quads_CMB_dg60.state', '7quads_SNeIa_dg60.state', 
              'all_dg60.state', 'all_SNeIa_dg60.state']
statefile = statefiles[7]  # 7  8/9 5/6
print(statefile)


In [ ]:
lm = LensModel(statedir+statefile)
lm.obj_idx = objidx
print(lm.__v__)


## Create synthetic images

In [ ]:
# lo.data = np.fliplr(lo.data)
reconsrc = ReconSrc(lo, lm, M=80, M_fullres=400, mask_keys=['circle'])
reconsrc.chmdl(-1)
sig2 = sigf * np.abs(reconsrc.lensobject.data)
sig2[sig2 == 0] = sig2[sig2 != 0].min()

print reconsrc.lensobject.data.shape


In [ ]:
# reconsrc.rotation = 1*reconsrc.lensobject.hdr['ORIENTAT']
# reconsrc.rotation = -95
# reconsrc.rotation = -4.0  # reconsrc.lensobject.hdr['ORIENTAT']
# reconsrc.rotation = -4.5  # reconsrc.lensobject.hdr['ORIENTAT']

In [ ]:
dij = reconsrc.lens_map(mask=True)
lmax = 0.0100 * np.max(dij)
plt.imshow(dij, extent=reconsrc.extent, cmap='gravic', origin='lower', interpolation='bicubic', vmin=0, vmax=lmax)
# plt.colorbar()
#plt.xlim(-3, 3)
#plt.ylim(-3, 3)
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')
plot_scalebar(R=reconsrc.lensobject.mapr, length=1)#, padding=(0.55, 0.55), barheight=0.03*0.5)
plot_labelbox(lens, position='top left')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.show()

In [ ]:
reconsrc.calc_psf("psf/tinytim_ACS.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_ACSF555W.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_ACSF814W.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_WFC3.fits", normalize=True, window_size=8, verbose=True)
# reconsrc.calc_psf("psf/tinytim_SBC.fits", normalize=True, window_size=8, verbose=True)

In [ ]:
kw = dict(method='minres', use_psf=False, use_mask=True, use_filter=False, sigma2=sig2.copy(),
          cached=True)


In [ ]:
%%script false
wrad = 0.8  # 0.5

reconsrc.chmdl(-1)
reconsrc.inv_proj_matrix(use_mask=False)
dij = reconsrc.lens_map(mask=True)
# dij = np.fliplr(dij)
src = reconsrc.plane_map(**kw)
synth = reconsrc.reproj_map(from_cache=False, save_to_cache=False, **kw)
# synth = np.fliplr(synth)
residmap = reconsrc.residual_map(nonzero_only=True, within_radius=wrad, from_cache=False, save_to_cache=False, **kw)
chi2 = reconsrc.reproj_chi2(reduced=False, nonzero_only=True, within_radius=wrad, from_cache=False, save_to_cache=False, **kw)
print(chi2)
print(np.sum(residmap))


lmax = 0.0150 * np.max(dij)      # 0.2500, 0.0075, 0.0075, 0.1000, 0.4000, 0.1000, 0.0100,  0.1000
sbf = (reconsrc.lensobject.px2arcsec[0]/reconsrc.src_pxscale)**2


plt.imshow(dij, extent=reconsrc.extent, cmap='gravic', origin='lower', interpolation='bicubic', vmin=0, vmax=lmax)
# plt.colorbar()
# plt.xlim(-3, 3)
# plt.ylim(-3, 3)
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')
plot_scalebar(R=reconsrc.lensobject.mapr, length=1, padding=(0.55, 0.55), barheight=0.03*0.5)
plot_labelbox(lens, position='top left')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_data.pdf".format(lens), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()



plt.imshow(src, extent=reconsrc.src_extent, cmap='gravic', origin='lower', interpolation='bicubic', vmin=0, vmax=lmax*sbf/1)
# plt.colorbar()
plot_scalebar(R=reconsrc.r_max, length=1)
plot_labelbox(lens, position='top left')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_src.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()



plt.imshow(synth, extent=reconsrc.extent, cmap='gravic', origin='lower', interpolation='bicubic', vmin=0, vmax=lmax)
# plt.colorbar()
# plt.xlim(-3, 3)
# plt.ylim(-3, 3)
plot_scalebar(R=reconsrc.lensobject.mapr, length=1, padding=(0.55, 0.55), barheight=0.03*0.5)
plot_labelbox(lens, position='top left')
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_synth.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()



plt.imshow(residmap, extent=reconsrc.extent, cmap='vilux', origin='lower')
# plt.xlim(-3, 3)
# plt.ylim(-3, 3)
plot_scalebar(R=reconsrc.lensobject.mapr, length=1)#, padding=(0.55, 0.55), barheight=0.03*0.5)
plot_labelbox(lens, position='top left')
c = reconsrc.lensobject.roi.buffer.center
c = (c - reconsrc.lensobject.center.xy) * reconsrc.lensobject.px2arcsec[0]
plt.plot(*c, marker='+', markersize=10, color='grey')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig("results/{}_{}_resid.pdf".format(lens, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()



In [ ]:
%%script false
qs = []
for i in range(0, reconsrc.model.N):
    reconsrc.chmdl(i)
    reconsrc.inv_proj_matrix(use_mask=False)
    chi2 = reconsrc.reproj_chi2(reduced=False, nonzero_only=True, within_radius=0.5, from_cache=False, save_to_cache=False, **kw)
    qs.append(chi2)
    print "{:04d}".format(i),
    # message = "{:8.4f} @ {:04d}".format(chi2, i)
    # print(message, end="\r")


In [ ]:
%%script false
np.savetxt("{}_{}_chi2.txt".format(lm.filename.replace('.state', ''), lens), qs)


In [ ]:
# %%script false
qs = np.loadtxt("results/7quads_dg60_B1608+656_chi2.txt")
# qs = np.loadtxt("results/7quads_dg60_DESJ0408-5354_chi2.txt")
print(len(qs))


In [ ]:
iqs = np.argsort(qs)
iq = iqs[0]
print iq, qs[iq]

In [ ]:
def H02aHz(H0arr):
    H0arr = np.asarray(H0arr)  # assume in units of km/s/Mpc
    H0 = H0arr.copy()
    H0 /= 3.08567758e1         # aHz=1e-18 sec
    return H0

def aHz2H0(vH0):
    vH0 *= 3.08567758e1
    return vH0

def H02Gyrs(H0arr):
    H0arr = np.asarray(H0arr)    # assume in units of km/s/Mpc
    invH0 = 1./H0arr.copy()      # s Mpc/km
    invH0 *= 3.08567758e3/np.pi  # Gyrs (Mpc=3.08567758e22 m; Gyr=pi*1e16 sec; km=1e3 m)
    return invH0

def H02critdens(H0arr):
    c = 299792458.0            # m s^-1
    G = 6.67430e-11            # kg^-1 m^3 s^-2
    GeV = 1.602176634e-10      # J/GeV
    H0arr = np.asarray(H0arr)  # assume in units of km/s/Mpc
    H02 = H0arr.copy()**2
    H02 /= 3.08567758e19*3.08567758e19  # s^-2
    c2_8piG = 3*c**2/(8*np.pi*G) # kg m^2 m^-3
    rhocrit = H02 * c2_8piG    # J / m^3
    rhocrit /= 1.602176634e-10 # GeV / m^3
    return rhocrit



In [ ]:
H0data = np.array(lm.H0)
H0data = H0data[iqs[:300]]  # best 30%

aHzH0 = H02aHz(H0data)
print(aHzH0[0])
invH0 = H02Gyrs(H0data)
print(invH0[0])
rhocrit = H02critdens(H0data)
print(rhocrit[0])



In [ ]:
q = np.percentile(aHzH0, [16, 50, 84])

fig, ax = plt.subplots()
axc = ax.twiny()

# second unit axis
def convertaxc(ax):
    x1, x2 = ax.get_xlim()
    axc.set_xlim(aHz2H0(x1), aHz2H0(x2))
    axc.figure.canvas.draw()
ax.callbacks.connect("xlim_changed", convertaxc)

n, bins, patches = ax.hist(aHzH0, bins=25, density=True, rwidth=0.901)
cm = plt.cm.get_cmap('phoenix')
ax.axvline(q[1], color=gcl.pink)
ax.axvline(q[0], color=gcl.red)
ax.axvline(q[2], color=gcl.red)

# add gaussian color scheme to pdf
bin_centers = 0.5 * (bins[:-1] + bins[1:])
x = np.exp(-(bin_centers - q[1])**2/np.mean(np.diff(q))**2)
x = x/x.max()
for c, p in zip(x, patches):
    plt.setp(p, 'facecolor', cm(c))
# add result text
plt.rcParams['mathtext.fontset'] = 'stixsans'
Hstr = 'H$_0$ = ${:5.1f}^{{{:+4.1f}}}_{{{:+4.1f}}}$'
q = np.percentile(H0data, [16, 50, 84])
plt.text(0.02, 0.85, Hstr.format(q[1], np.diff(q)[1], -np.diff(q)[0]), fontsize=19, color='black', transform=plt.gca().transAxes)

fig.axes[0].get_yaxis().set_visible(False)
ax.set_xlabel('H$_0$ [aHz]')
axc.set_xlabel('H$_0$ [km/s/Mpc]', fontsize=12)
plt.setp(axc.get_xticklabels(), fontsize=12)
plt.tight_layout()
plt.savefig('results/_H0hist_{}_filter{}.pdf'.format(statefile.replace('.state', ''), lens), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()